In [ ]:
"""
Coastline Generator:
This code generates a 2D fractal coastline given a fractal dimension between(1,2)

Cardinal Code: Entrance/Exit Direction number code
 _____
|  1  |
|4   2|
|_ 3__|


"""

from pylab import *
from numpy import *
import collections
import PIL
from PIL import ImageDraw


'''---------------------------
DEFINE SHORT FUNCTIONS
---------------------------'''
def paintcoast(Matrix):
    a = PIL.Image.new('RGB', Matrix.shape, color=(154,174,255))

    for j in xrange(Matrix.shape[0]):
        for i in xrange(Matrix.shape[0]):
            if Matrix[j][i]==1:
                a.putpixel((i,j),(128,70,35))
    ImageDraw.floodfill(a,(1023,1023),(128,70,35))
    return a


class coast:
    def create_grid(self, base, endpos):
        # Coordinates of endpos
        end_col = endpos[0]
        end_row = endpos[1]
        # Create the 2 types of strings
        string_normal = "."*base
        string_end = "."*end_col + "*" + "."*(base-end_col-1) # replace one "." by "*" to represent endpos
        # Combine them to create the grid
        grid = [string_normal]*(end_row) + [string_end] + [string_normal]*(base-end_row-1)
        #grid_basic = [string]*base
        return grid

# Function bfs(,)
#   Computes shortest path between two cells, using a "breadth first search" algorithm
#   input = grid, starting position, end position
#   output = list of cells visited in shortest path, in order
#   Source: https://stackoverflow.com/questions/47896461/get-shortest-path-to-a-cell-in-a-2d-array-python
    def bfs(self,grid, start, base): #start = (col, row)
        wall, clear, goal = "#", ".", "*"
        width, height = base, base
        queue = collections.deque([[start]]) # deque: Build an ordered collection with optimized access from its endpoints.
        seen = set([start])
        while queue:
            path = queue.popleft() # Remove and return the leftmost element
            x, y = path[-1]
            if grid[y][x] == goal: # if we are at the exit cell ("endpos")
                return path
            for x2, y2 in ((x+1,y), (x-1,y), (x,y+1), (x,y-1)):
                if 0 <= x2 < width and 0 <= y2 < height and grid[y2][x2] != wall and (x2, y2) not in seen:
                    queue.append(path + [(x2, y2)])
                    seen.add((x2, y2))


# FUNCTION walking
# INPUT: base,sqnum,entrance,exit
# Ouput: "path", in format: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3)]
    def walking(self, base,sqnum,entrance,walkexit):
        print "walking"
        ocean = zeros((3,base,base))  # (z = depth, x = rows, y = cols)
        initpos = [0,0]

        # 1.   Select a random entrance [col,row]
        if type(entrance)==str:
            if entrance=='n':
                initpos = [randint(0,base-1),0]
            elif entrance=='e':
                initpos = [base-1,randint(0,base-1)]
            elif entrance=='w':
                initpos = [0,randint(0,base-1)]
            elif entrance=='s':
                initpos = [randint(0,base-1),base-1]
        else:
            initpos[0] = entrance[0]
            initpos[1] = entrance[1]

        # 2.   Find a path such that:
        #     - sqnum is satisfied (with error or +/-1)
        #     - exit cell ("endpos") is indeed on the side of the exit ('n', 'e', 'w' or 's')

        path_over = 0
        step_while = 0

        while (path_over == 0) and (step_while < 5): # we settle for a "wrong" path (i.e. nb of cells visited is way too large or small) after 5 failed attempts

            # a.  Select a random endpos which is not equal to the initpos position:
            print "pathseeking"
            endpos = initpos
            while initpos[0] == endpos[0] and initpos[1] == endpos[1]:
                print walkexit
                if walkexit=='n':
                    endpos = [randint(0,base-1),0]
                elif walkexit=='e':
                    endpos = [base-1,randint(0,base-1)]
                elif walkexit=='w':
                    endpos = [0,randint(0,base-1)]
                elif walkexit=='s':
                    endpos = [randint(0,base-1),base-1]
            # --> endpos = [end_col,end_row]. Ex: with base = 4, [3,3] = bottom-right cell

            # b.  Create string GRID
            grid = self.create_grid(base, endpos)
            # c.  Compute shortest path
            start = tuple(initpos)
            print grid,start,base
            path = self.bfs(grid, start, base) # type(path) --> "list"

            # d.  If nb of cells visited by that path is = sqnum +/-1, we are done! Otherwise, start agin
            if (len(path) == sqnum) or (len(path) == sqnum+1) or (len(path) == sqnum-1):
                path_over = 1
            else:
                path_over = 0

            step_while += 1
        # end of the while loop.  Now we have the path, in format: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3)]
        print "pathfound"
        return path



    def pastepath(self,patharray,formerpath,base):
        #Creates the master path after finishing each scale level recursion
        masterpath = []
        for i in xrange(len(patharray)):
            for j in xrange(len(patharray[i])):
                masterpath.append([sum(x) for x in zip(patharray[i][j],[k*base for k in formerpath[i]])]) #[k*base for k in formerpath[1]]
                #masterpath
        return masterpath


    def mappingfunc(self,path,width):
        # Creates a square matrix from a path
        M = zeros((width,width))
        #print M
        for i in xrange(len(path)):
            j = path[i][0]
            k = path[i][1]
            if type(k)==list:
                k=int(k[0])
            if type(j)==list:
                j=int(j[0])
            #print j,k
            M[path[i][1]][path[i][0]]=1
        return M

#This function maps every result into the main matrix
    def finddirection(self,path,nelement):
        distance = list(subtract(path[nelement],path[nelement+1]))
        #print "distance "+str(distance)
        direction = ''
        if distance == [0,-1]:
            direction = 's'
        elif distance == [0,1]:
            direction = 'n'
        elif distance == [1,0]:
            direction = 'w'
        elif distance == [-1,0]:
            direction = 'e'
        print "direction: "+direction
        return direction


#This function plots the final path
    def plot_final_path(self,Fd,base,Ns,width,height,path):
        # Final matrix - initially is zeros
        M = zeros((height,width))
        # Replace every colored cell by 1
        for i in range(0,len(path)):
            icol = path[i][0]
            irow = path[i][1]
            M[irow,icol] = 1
        # Plot M
        #   spy(Z) plots the sparsity pattern of the 2-D array Z
        fig_coast = spy(M,markersize=5)
        string_1 = 'fractality = %1.1f, ' % (Fd)
        string_2 = 'approximation = %d levels)' % (Ns)
        title('Coastline (' + string_1 + string_2)
        '''
        # if want to save as .png
        savefig('coastline_1.png')
        '''
        return fig_coast

    def paintlandscape(self,Matrix):
        a = PIL.Image.new('RGB', Matrix.shape, color=(154,174,255))
        painting = False
        for j in xrange(Matrix.shape[0]):
            paint = False
            s = sum(Image[j][:])
            error = s%2==0
            c=0
            for i in xrange(Matrix.shape[0]):
                if Matrix[j][i]==1:
                    c+=1
                    if c==1 and error:
                        a.putpixel((i,j),(128,70,35))
                    else:
                        painting = not painting
                if painting:
                    a.putpixel((i,j),(128,70,35))
        return a



# randint(1,150)
    ##################################
    #    Loop Creating a Fractal     #
    ##################################
    # Here's where the magic happens

    def create(self):
        base = self.base
        nbox = self.nbox
        globalentrance = self.globalentrance
        globalexit = self.globalexit
        formerpath = self.walking(base,nbox,globalentrance,globalexit)
        masterpath = formerpath
        Ns = self.Ns
        #print self.mappingfunc(formerpath,base)
        for i in xrange(int(Ns)-1):#Not sure of this -1 yet
            patharray = []
            print "depth level %i"%(i)
            for j in xrange(len(formerpath)):
                print "path number %i"%(i)
                if j==0:
                    entrance = globalentrance
                else:
                    print "indirection"
                    lastexit=self.finddirection(formerpath,j-1)
                    print lastexit
                    if lastexit == 'n':
                        entrance =  add(patharray[j-1][-1],[0,base-1])
                    elif lastexit == 's':
                        entrance =  add(patharray[j-1][-1],[0,1-base])
                    elif lastexit == 'e':
                        entrance =  add(patharray[j-1][-1],[1-base,0])
                    elif lastexit == 'w':
                        entrance =  add(patharray[j-1][-1],[base-1,0])
                    print entrance

                if j==(len(formerpath)-1):
                    exit = globalexit
                else:
                    #print "outdirection"
                    exit = self.finddirection(formerpath,j)
                    #print exit
                print "walking "
                print entrance
                print exit
                path = self.walking(base,nbox,entrance,exit)
                #print path
                patharray.append(path)
            #print patharray
            formerpath =self.pastepath(patharray,formerpath,base)
            del patharray
            #print self.mappingfunc(formerpath,base**(i+2))
        finalpath = formerpath
        Image = self.mappingfunc(finalpath,self.width)
        return Image

    def  __init__(self,base,width,Fd):
        self.width = width  #Final image Width
        self.height = width # Final image height
        self.base = base  # Base for dividing the image
        self.Ns = log(width)/log(base) # Number of scales we are going to work with
        self.ocean = zeros((width,width))
        self.Fd = Fd # Fractal Dimension
        #Entrance and exit from the big picture. They can't be equal (See cardinal format at the beginning of this code)
        self.globalentrance = 'n'
        self.globalexit = 's'
        self.nbox = int(round(base**Fd))


######################################################################################
#                                    MAIN
######################################################################################

if __name__ == '__main__':
    ############################
    #       Parameters         #
    ############################
    base = int(raw_input("Image Base"))
    side = int(raw_input("Image Side"))
    Fd = float(raw_input("Fractal Dimension"))

    coastline = coast(base,side,Fd)
    result = coastline.create()
    Image = paintcoast(result)



# [for (5, 2), (6, 2), (6, 1), (6, 0)]


In [ ]:
Image

In [13]:
Image.save("coastlineImage.png","PNG")